| | |
|---|---|
| **Name** | Jan Ubbo van Baardewijk & Thomas Brus |
| **Group** | ML_HMI_01 |


In [2]:
import numpy as np
import operator

# Decision Trees

In this lab, we will implement the C4.5 algorithm for classification with decision trees. 


## Data

Let's start by loading the datasets that we will be working with. The ```mushroom``` dataset is a dataset describing the features of different species of mushrooms, and the goal is to derive simple rules to predict whether a species is poisonous or not.

In [3]:
# This code is given
data = np.load("uci-mushrooms.npz")
print data.keys()

d = data['inputs']
l = data['targets']
fname = data['feature_names']
print len(fname)

['inputs', 'targets', 'feature_names']
22


## Helper functions

Here are a few functions to deal with categorical labels

In [18]:
# This code is given

def listValues(inputlist):
    """List the possible values of a given feature in a dataset"""
    values = set([])
    for x in inputlist:
        values.add(x)
    return list(values)

def countLabels(labelList):
    """Return the most common label in a list"""
    counts = {}
    for x in labelList:
        if x not in counts:
            counts[x] = 1
        else:
            counts[x] += 1
    return counts

def majorityLabel(labelList):
    counts = countLabels(labelList)
#    print "Majority label:", counts
    return max(counts.iteritems(), key=operator.itemgetter(1))[0]

print listValues(l)
print listValues(d[:,1])
print majorityLabel(d[:,1])

['POISONOUS', 'EDIBLE']
['FIBROUS', 'GROOVES', 'SMOOTH', 'SCALY']
SCALY
8416


## Computing the entropy of a list of categorical values

The decision tree makes its decision based on decision nodes, and these nodes are evaluated based on information gain they provide on the training data. Information gain is computed as 
$$
IG = H(parent) - \sum_j \frac{N_j}{N} H(child=j)
$$
the entropy of the data associated with the parent, minus the weighted entropy of the data-subsets associated with the chidlren

**question 1 [5 credits]]** Implement a helper function that computes the entropy of a set of categorical labels


In [27]:
from math import log
from __future__ import division

def entropy(labelList):
    edibleCount = len([label for label in labelList if label == "EDIBLE"])
    poisonousCount = len([label for label in labelList if label == "POISONOUS"])
    ediblePercentage = edibleCount / len(labelList)
    poisonousPercentage = poisonousCount / len(labelList)

    return -ediblePercentage * log(ediblePercentage, 2) - poisonousPercentage * log(poisonousPercentage, 2)

print entropy(l)

0.996803828522


## Implementing the tree

Below, you are given the code to make a decision tree, train it and classify datapoints using binary decision nodes based on categorical variables. The implementation includes  ```Leaf``` nodes, wich return a classification result, and ```BinaryNode```, which decide which child should be responsible for the further classification.

The ```BinaryNode``` constructor selects the most informative value in the data it's trained on, and makes a decision based on whether or not the feature of the datapoint to be classified equals that value, or not.

In [ ]:
# This code is given

class Node:
    def classify(self, datapoint):
        """Return the classification label for the given datapoint"""
        raise Exception("Not Implemented")        
    def entropy(self):
        """Return the entropy of the training data associated with this node"""
    def informationGain(self):
        """Return the information gain of this node on the training data """
        raise Exception("Not Implemented by derived class")

class Leaf(Node):
    def __init__(self,inputs, targets):
        self.v = majorityLabel(targets)
        self.inputs = inputs
        self.targets = targets
        self.children = None
        self.H = entropy(targets)
        
    def entropy(self):
        return self.H
    
    def informationGain(self):
        return 0.
    
    def classify(self,datapoint):
        return self.v
    
    def __str__(self):
        return "\t\t\t=> %s" % (self.v)
    
class BinaryNode(Node):
    """Make a decision based on whether a given feature equals a given value, or not"""
    def __init__(self, inputs, targets, feature):
        self.feature = feature
        self.value   = None
        self.IG      = 0.0
        self.H       = entropy(targets)                     # Entropy of unsplit data
        self.inputs  = inputs
        self.targets = targets
        N = float(targets.size)                             # Total number of datapoints

        for value in listValues(inputs[:,feature]):
            indicesTrue  = inputs[:,feature]==value
            indicesFalse = inputs[:,feature]!=value

            if indicesTrue.all() or indicesFalse.all():
                continue

            children = [ Leaf(inputs[indicesTrue,:], targets[indicesTrue]),
                         Leaf(inputs[indicesFalse,:], targets[indicesFalse]) ]
            conditions = [ "%s==%s" % (fname[feature], value), "%s!=%s" % (fname[feature], value) ]

            Nt = float(children[0].targets.size)        # N.o. datapoints in True condition
            Nf = float(children[1].targets.size)        # N.o. datapoints in False condition
            pt = Nt/N
            pf = Nf/N
            ig = self.H - pt * children[0].entropy() - pf * children[1].entropy()
            
            if ig > self.IG:
                self.value = value
                self.IG = ig
                self.children = children
                self.conditions = conditions
    
    def entropy(self):
        return self.H
    
    def informationGain(self):
        """Return the entropy gain of splitting the given data according to 
        the specified feature and value"""
        return self.IG
    
    def classify(self, x):
        if x[self.feature] == self.value:
            return self.children[0].classify(x)
        else:
            return self.children[1].classify(x)
        
    def __str__(self):
        return " => Binary decision: '%s =?= %s'" % (fname[self.feature], self.value )
    
    

In [ ]:
# This code is given

def c45(inputs, targets, minSize=0):
    ig = 0.
    res = None
    for feature in range(inputs.shape[1]):
        n = BinaryNode(inputs,targets,feature)
        if n.informationGain() > ig:
            ig = n.informationGain()
            res = n
    for i in range(len(res.children)):
        if res.children[i].entropy() > 0.:
            res.children[i] = c45(res.children[i].inputs, res.children[i].targets, minSize)
    return res

root = c45(d,l)

In [ ]:
# This code is given

def prettyPrint(tree, cond="", dlist = [True]):
    indent = ""
    for d in dlist[:-1]:
        indent += "   " if d else "|  "
    print indent + "+-> " + cond + " " + tree.__str__()
    if tree.children:
        for cond,cld in zip(tree.conditions[:-1],tree.children[:-1]):
            prettyPrint(cld,cond,dlist + [ False ])
        prettyPrint(tree.children[-1], tree.conditions[-1], dlist+[ True ])

    

prettyPrint(root)


# Training 

The code above trains a decision tree.

**Question 2 [5 credits]** Split the ```mushroom``` dataset into 10 folds and perform 10-fold cross-validated training and testing of the decision tree. Report de tree's performance in terms of average precision, recall and accuracy.

**Question 3 [5 credits]** Modify the training function provided above to stop splitting nodes when the corresponding training data contains less than ```minSize``` datapoints, where ```minSize``` is a parameter to the function. Split the data into 80\% train and 20\% test, and use 10-fold cross-validation on the training set to find a good value for this parameter. Then report the confusion matrix, precision, recall and accuracy that you obtain on the test set.


In [ ]:
# Answer to Q2

In [ ]:
# Answer to Q3

# Different types of decisions

Our tree for now can only make decisions based on binary conditions, but our dataset contains features with more than two possible values. 

**Question 4 [10 credits]** Implement a new type of node, ```MultiNode``` that has a child per possible value of the feature. Be careful to deal with the possibility that the test data may contain categories that the training set didn't have.

In [ ]:
# This code must be modified

class MultiNode(Node):
    """Make a decision based the value of the given feature, i.e. as many children as there are 
    possible values for the feature"""
    def __init__(self, inputs, targets, feature):
        # YOUR CODE COMES HERE
        
                
    def entropy(self):
        return self.H
    
    def informationGain(self):
        """Return the entropy gain of splitting the given data according to 
        the specified feature and value"""
        return self.IG
    
    def classify(self, x):
        # YOUR CODE COMES HERE
        
    def __str__(self):
        return "=> n-ary decision: %s = ?" % (fname[self.feature]) # You may want to change this to suit your own implementation
    

## Dealing with heterogeneous features

The code below keeps a list of the type of node that is applicable for each feature, in this case ```BinaryNode``` or ```Multinode```. 

**Question 5 [5 credits]** Is a tree of multinodes better than a tree of binary nodes? Is it more compact? Is it generalising better?

*Answer to Q5*

In [ ]:
#This code is given but minSize is not implemented

featureType = [ MultiNode for i in range(l.size) ]

def c45(inputs, targets, minSize, featureType):
    ig = 0.
    res = None
    for feature in range(inputs.shape[1]):
        n = featureType[feature](inputs,targets,feature)
        if n.informationGain() > ig:
            ig = n.informationGain()
            res = n
    for i in range(len(res.children)):
        if res.children[i].entropy() > 0.:
            res.children[i] = c45(res.children[i].inputs, res.children[i].targets, minSize, featureType)
    return res

root = c45(d,l,5, featureType)
prettyPrint(root)

In [ ]:
# Playground for answering Q5

## Continuous values

Until now, we have only dealt with categorical features. In the next part of the lab, you will deal with mixed categorical and continuous features.

One way to deal with continuous features is to split the dataset based on whether the feature is larger than a certain threshold, or not. The number of possible threshold values to consider is, of course, infinite in theory, but when deciding what the threshold value should be, based on a training dataset, only threshold values that will result in a different classification of those training examples matter: those are values that are smaller than one datapoint's feature value and larger than another's. Moreover, when considering a threshold in between two feature values, any value of the threshold in that range will result in the same classification, but to maximise generalisation it makes sense to maximise the margin: that is, to take a threshold value that is in the middle between the two.

Also note that it only makes sense to consider ranges of feature values that are disjoint, i.e., consider thresholds that are in the middle between neighbouring datapoints (along that feature's dimension), not in the middle between any pair of datapoints.

**Question 6 [10 credits]**. You guessed it, implement a node ```ContNode``` that selects a threshold value based on its training subset and classifies new datapoints based on whether they are smaller than that value, or not.

In [ ]:
# This code must be modified.

class ContNode(Node):
    """Make a binary decision based on a threshold, for the value of the given feature"""
    def __init__(self, inputs, targets, feature):
         # YOUR CODE COMES HERE
                
    def entropy(self):
        return self.H
    
    def informationGain(self):
        """Return the entropy gain of splitting the given data according to 
        the specified feature and value"""
        return self.IG
    
    def classify(self, x):
        if x[self.feature] >= self.value:
            return self.children[0].classify(x)
        else:
            return self.children[1].classify(x)
        
    def __str__(self):
        return "=> threshold decision: %s >= %f" % (fname[self.feature],self.value)
    

## Classification based on mixed features

The following dataset contains 14 features describing people's situation, and the classification task is to predict whether they earn more than \$50k a year or not. The categorical decision nodes we implemented above can work with any representation of categorical data, but the continuous decision node requires a numerical representation of the feature. Numpy arrays cannot easily contain both strings and numbers, and so to simplify our life, we converted the dataset to 

To allow our classifier to work with mixed features, we use a numerical representation of the categorical features in the following. So, let's first load the data:

In [ ]:
# This code is given

income = np.load('income.npz')
print income.keys()
inputs = income['inputs']
targets = income['targets']
fname = income['fnames']
ftypes = income['ftypes']

**Question 7 [10 credits]** Take 1000 training datapoints at random from the dataset, and another 1000 test points that are distinct from those (don't mix train and test). Train a decision tree on the train set and report the confusion matrix obtained on the test set. What are the obtained precision, recall and accuracy? 

In [ ]:
# This code is given

featureType = [ ContNode if ftypes[i]=='continuous' else BinaryNode for i in range(len(ftypes)) ]
tree = c45(inputs[:10000,:],targets[:10000],100,featureType)
prettyPrint(tree)

# Answer to Q7
